In [1]:
import pandas as pd
import nltk
pd.set_option('display.max_colwidth', -1)

In [2]:
train_data=pd.read_csv("train.csv",sep="~")
train_data.head()

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,11755,"After reading mixed reviews I almost didn't book at the W, but I was attending a concert at the Pantages Theatre and you can't get any more convenient than the W. For those using mass transit, it's literally on top of the Hollywood and Vine station - take the Red Line from Union Station.\r\nATTITUDE: yes, this is a new and very hip hotel. But every staff member I met was friendly and totally committed to service. From the valet parking to the front desk to room service to the Men in Black (security folks needed for the club scene), everyone was great.\r\nFOOD: I always sample room service my first time at any property. Food arrived earlier than promised and I was impressed. The onion soup gratinee was simply the best I've ever had. A simple prime beef cheeseburger was great: a hand formed patty in a light bun with the usual trimmings and condiments. And the French fries... I've never been impressed by fries before but now I am.\r\nROOM: Trendy, well appointed and comfortable. The universal cell phone (or other gizmo) charger was handy. If you're worried about party noise from the clubs, spend a few extra bucks for an exterior-facing room. I did and had no trouble with noise.\r\nSUMMARY: Would I come here again? Absolutely.",Google Chrome,Desktop,Good
1,33912,"This motor inn is located about - city blocks from Fisherman's Warf and about - blocks from the Embarcadero. You are in North Beach and walking distance from China Town and all city transportation. The hotel itself is small and clean. It is old and not updated, but the beds were still good and the room was well cleaned daily. My only complaint would be the bathroom space being a bit small for our family. There was a great, large, flat screen (not that we were there very often)! The staff was extremely accommodating and helpful with recommendations and directions. There is free parking here, which is amazing!",Firefox,Tablet,Good
2,10143,It was our first time there and surely not our last. Arrived very early off the train and went there to drop off our luggage but to our surprise the room was already ! The rooftop bar ia certainly beautiful. The hospitality within the hotel superb ! I admireed the european breakfast but wished they had more healthy choices. Perhaps a pan of scrambled eggs instead of all the pastries and rolls. Too many carbs and not enough protein. Wish there ahad been a coffe maker in the room too. See you all soon.,Google Chrome,Mobile,Good
3,33114,"Great hotel in an excellent location, just off Times Square and next door to the Nederlander Theatre (home of the musical Rent). Rooms very clean and lovely fresh muffins at breakfast!\r\nBeware if your room is on the first floor, we could hear the noise from the hotel bar below. It does close at a reasonable hour,but if you need an early night after a hard day sightseeing then ask for a high floor cos it did keep us awake.",Mozilla,Desktop,Good
4,17464,We stayed at the hotel for - weeks to get away from a very long Massachusetts winter. The hotel is on an island with a waterway in front and a marina in the back. Our room had a table so we could play cards eat etc. with the water view. \r\nThe only thing negative at all is the light was a little dim in the room.\r\nThe staff is very friendly & helpful. They were always willing to recommend restaurants and gave us coupons to the many attractions that San Diego has to offer. They let us use beach bikes to explore the island. There is little or no traffic there and a trail that is on the other side of the marina.\r\nIt was a great vacation!\r\nWe will be back!,Google Chrome,Desktop,Good


In [3]:
train_data.drop(['User_ID','Browser_Used','Device_Used'],inplace=True,axis=1)

In [4]:
train_data.head()

,Description,Is_Response
0,"After reading mixed reviews I almost didn't book at the W, but I was attending a concert at the Pantages Theatre and you can't get any more convenient than the W. For those using mass transit, it's literally on top of the Hollywood and Vine station - take the Red Line from Union Station.\r\nATTITUDE: yes, this is a new and very hip hotel. But every staff member I met was friendly and totally committed to service. From the valet parking to the front desk to room service to the Men in Black (security folks needed for the club scene), everyone was great.\r\nFOOD: I always sample room service my first time at any property. Food arrived earlier than promised and I was impressed. The onion soup gratinee was simply the best I've ever had. A simple prime beef cheeseburger was great: a hand formed patty in a light bun with the usual trimmings and condiments. And the French fries... I've never been impressed by fries before but now I am.\r\nROOM: Trendy, well appointed and comfortable. The universal cell phone (or other gizmo) charger was handy. If you're worried about party noise from the clubs, spend a few extra bucks for an exterior-facing room. I did and had no trouble with noise.\r\nSUMMARY: Would I come here again? Absolutely.",Good
1,"This motor inn is located about - city blocks from Fisherman's Warf and about - blocks from the Embarcadero. You are in North Beach and walking distance from China Town and all city transportation. The hotel itself is small and clean. It is old and not updated, but the beds were still good and the room was well cleaned daily. My only complaint would be the bathroom space being a bit small for our family. There was a great, large, flat screen (not that we were there very often)! The staff was extremely accommodating and helpful with recommendations and directions. There is free parking here, which is amazing!",Good
2,It was our first time there and surely not our last. Arrived very early off the train and went there to drop off our luggage but to our surprise the room was already ! The rooftop bar ia certainly beautiful. The hospitality within the hotel superb ! I admireed the european breakfast but wished they had more healthy choices. Perhaps a pan of scrambled eggs instead of all the pastries and rolls. Too many carbs and not enough protein. Wish there ahad been a coffe maker in the room too. See you all soon.,Good
3,"Great hotel in an excellent location, just off Times Square and next door to the Nederlander Theatre (home of the musical Rent). Rooms very clean and lovely fresh muffins at breakfast!\r\nBeware if your room is on the first floor, we could hear the noise from the hotel bar below. It does close at a reasonable hour,but if you need an early night after a hard day sightseeing then ask for a high floor cos it did keep us awake.",Good
4,We stayed at the hotel for - weeks to get away from a very long Massachusetts winter. The hotel is on an island with a waterway in front and a marina in the back. Our room had a table so we could play cards eat etc. with the water view. \r\nThe only thing negative at all is the light was a little dim in the room.\r\nThe staff is very friendly & helpful. They were always willing to recommend restaurants and gave us coupons to the many attractions that San Diego has to offer. They let us use beach bikes to explore the island. There is little or no traffic there and a trail that is on the other side of the marina.\r\nIt was a great vacation!\r\nWe will be back!,Good


In [5]:
from nltk.corpus import stopwords
from string import punctuation
def review_to_words(review):
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", review.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    return words

In [6]:
train_data.Description[10]

'My mom and I stayed here for - nights on a short vacation to Seattle and were extremely happy with our accommodations here.\r\nCheck in was a breeze (if only our car rental had gone as smoothly) and the front desk clerk was cordial and friendly, giving us suggestions for attractions and dining nearby.\r\nThe room was clean and comfortable, and included a fridge, sink, coffee maker, and microwave in a pseudo """"""""""""""""""""""""""""""""bar"""""""""""""""""""""""""""""""" area. They even included paper plates and utensils. \r\nFront desk and guest services were cordial and helpful, bringing me a full service set of extra coffee, mugs, spoons, sugar, half-half (which I suspect they borrowed from their on-site restaurant, Emerald Grill), when I only asked for extra packets of coffee and cream!\r\nOne of the best things about this hotel was the full business center, open ---hours, which is FREE to hotel guests. It was spacious (not like the closet-sized ones I\'ve used in other hotels)

In [7]:
review_to_words(train_data.Description[1])

NameError: name 're' is not defined